## Predict historical prices of Brent Oil, Crude Oil WTI, Natural Gas, Heating Oil from 2000-2022 with Fedot.Industrial

Dataset published on Kaggle3 consists of historical prices of Brent Oil, CrudeOil WTI, Natural Gas, and Heating Oil from 2000 to 2022. This sample of DailyOilGasPrices was created by using 30 consecutive business days of **Crude Oil WTI close prices** and **traded volumes** as **predictors** and the **average natural gas close** price during each 30-day time frame as the **target** variable. The final dataset has 191 2-dimensional time series of length 30, of which 70% were randomly sampled as training data and the remaining 30% as testing data. This type of model could help companies and governments to better analyse and predict economic situations and correlations regarding oil and natural gas.
Link to the dataset - https://www.kaggl.com/datasets/prasertk/historical-daily-oil-and-natural-gas-prices

In [1]:
import pandas as pd
from fedot_ind.api.utils.path_lib import PROJECT_PATH
from fedot.core.pipelines.pipeline_builder import PipelineBuilder
from fedot_ind.tools.loader import DataLoader
from fedot_ind.api.main import FedotIndustrial

In [2]:
def evaluate_loop(api_params, finetune: bool = False):
    industrial = FedotIndustrial(**api_params)
    if finetune:
        industrial.finetune(train_data)
    else:
        industrial.fit(train_data)
    return industrial

The list of basic fedot industrial models for experiment are shown below. We using simple linear machine learning pipelines with 3 different feature generators: Statistical, Reccurence, Topological. And for each of them we add PCA transformation with 90 % of explained dispersion.

In [3]:
initial_assumption = PipelineBuilder().add_node('quantile_extractor').add_node('treg')
params = dict(problem='regression',
                             metric='rmse',
                             timeout=15,
                             initial_assumption=initial_assumption,
                             n_jobs=2,
                             logging_level=20)
dataset_name = 'DailyOilGasPrices'
data_path = PROJECT_PATH + '/examples/data'

Now we must download the dataset. It could be done by using `DataReader` class that implemented as attribute of `FedotIndustrial` class. This class firstly tries to read the data from local project folder `data_path` and then if it is not possible, it downloads the data from the UCR/UEA archive. The data will be saved in the `data` folder.

In [4]:
_, train_data, test_data = DataLoader(dataset_name=dataset_name).read_train_test_files(
            dataset_name=dataset_name,
            data_path=data_path)

2024-04-03 16:08:27,693 - Reading data from D:\WORK\Repo\Industiral\IndustrialTS/examples/data/DailyOilGasPrices


In [5]:
import numpy as np
features = np.array(train_data[0].values.tolist()).astype(float)

Lets check our data.

In [6]:
features.shape

(133, 2, 30)

Lets visualise our predictors.

In [30]:
from matplotlib import pyplot as plt
pd.DataFrame(features[1, 0, :]).plot(title='Crude Oil WTI close prices')
pd.DataFrame(features[1, 1, :]).plot(title='Crude Oil WTI traded volumes')
plt.show()

Next steps are quite straightforward. We need to fit the model and then predict the values for the test data just like for any other model in sklearn.

At the `fit` stage FedotIndustrial will transform initial time series data into features dataframe and will train regression model.

In [8]:
industrial_model =  evaluate_loop(api_params=params, finetune=True)

2024-04-03 16:10:18,852 - Initialising experiment setup
2024-04-03 16:10:18,855 - Initialising Industrial Repository
2024-04-03 16:10:18,971 - Initialising Dask Server
Creating Dask Server
2024-04-03 16:10:19,445 - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-04-03 16:10:19,478 - State start
2024-04-03 16:10:19,480 - Found stale lock file and directory 'C:\\Users\\user\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-dz5roxue', purging
2024-04-03 16:10:19,482 - Found stale lock file and directory 'C:\\Users\\user\\AppData\\Local\\Temp\\dask-scratch-space\\worker-er2nmzvc', purging
2024-04-03 16:10:19,490 -   Scheduler at: inproc://10.64.4.217/22788/1
2024-04-03 16:10:19,491 -   dashboard at:  http://10.64.4.217:8787/status
2024-04-03 16:10:19,492 - Registering Worker plugin shuffle
2024-04-03 16:10:19,505 -       Start worker at: inproc://10.64.4.217/22788/4
2024-04-03 16:10:19,506 -          Listeni

[I 2024-04-03 16:12:15,622] A new study created in memory with name: no-name-8b21e0dd-690c-43b1-b10d-73d0ca077743


  0%|          | 0/200 [00:00<?, ?it/s]

[I 2024-04-03 16:12:17,581] Trial 7 finished with value: 2.445611769767872 and parameters: {'0 || treg | max_features': 0.8877983365258869, '0 || treg | min_samples_split': 9, '0 || treg | min_samples_leaf': 10, '0 || treg | bootstrap': False}. Best is trial 7 with value: 2.445611769767872.
[I 2024-04-03 16:12:17,613] Trial 4 finished with value: 2.445611769767872 and parameters: {'0 || treg | max_features': 0.4825005163339129, '0 || treg | min_samples_split': 4, '0 || treg | min_samples_leaf': 7, '0 || treg | bootstrap': False}. Best is trial 7 with value: 2.445611769767872.
[I 2024-04-03 16:12:17,721] Trial 3 finished with value: 2.445611769767872 and parameters: {'0 || treg | max_features': 0.9129674550392047, '0 || treg | min_samples_split': 8, '0 || treg | min_samples_leaf': 15, '0 || treg | bootstrap': False}. Best is trial 7 with value: 2.445611769767872.
[I 2024-04-03 16:12:17,832] Trial 2 finished with value: 2.429942409634732 and parameters: {'0 || treg | max_features': 0.735

In [9]:
labels = industrial_model.predict(test_data)
probs = industrial_model.predict_proba(test_data)
metrics = industrial_model.get_metrics(target=test_data[1],
                                 rounding_order=3,
                                 metric_names=('r2', 'rmse', 'mae'))

In [10]:
metrics

,r2,rmse,mae
0,0.671,1.032,0.709


At the end of the experiment we can obtain the desired metric values using `calculate_regression_metric` method. Now there are five available metrics for classification task:
- `explained_variance_score`
- `max_error`
- `mean_absolute_error`
- `mean_squared_error`
- `d2_absolute_error_score`.
- `median_absolute_error`
- `r2_score`

## AutoML approach

In [11]:
industrial_auto_model = evaluate_loop(api_params=params, finetune=False)

2024-04-03 16:17:29,001 - Initialising experiment setup
2024-04-03 16:17:29,003 - Initialising Industrial Repository
2024-04-03 16:17:29,004 - Initialising Dask Server
Creating Dask Server
2024-04-03 16:17:29,022 - State start
2024-04-03 16:17:29,031 -   Scheduler at: inproc://10.64.4.217/22788/9
2024-04-03 16:17:29,032 -   dashboard at:  http://10.64.4.217:55185/status
2024-04-03 16:17:29,034 - Registering Worker plugin shuffle
2024-04-03 16:17:29,045 -       Start worker at: inproc://10.64.4.217/22788/12
2024-04-03 16:17:29,046 -          Listening to:          inproc10.64.4.217
2024-04-03 16:17:29,046 -           Worker name:                          0
2024-04-03 16:17:29,047 -          dashboard at:          10.64.4.217:55188
2024-04-03 16:17:29,047 - Waiting to connect to: inproc://10.64.4.217/22788/9
2024-04-03 16:17:29,048 - -------------------------------------------------
2024-04-03 16:17:29,048 -               Threads:                          8
2024-04-03 16:17:29,049 -     

Generations:   0%|          | 0/10000 [00:00<?, ?gen/s]

2024-04-03 16:17:29,861 - IndustrialDispatcher - Number of used CPU's: 2
2024-04-03 16:17:35,069 - IndustrialDispatcher - 1 individuals out of 1 in previous population were evaluated successfully.
2024-04-03 16:17:35,112 - IndustrialEvoOptimizer - Generation num: 1 size: 1
2024-04-03 16:17:35,113 - IndustrialEvoOptimizer - Best individuals: HallOfFame archive fitness (1): ['<rmse=1.885 node_number=0.200>']
2024-04-03 16:17:36,176 - IndustrialDispatcher - Number of used CPU's: 2
2024-04-03 16:19:56,027 - IndustrialDispatcher - 21 individuals out of 21 in previous population were evaluated successfully.
2024-04-03 16:19:56,122 - IndustrialEvoOptimizer - Generation num: 2 size: 21
2024-04-03 16:19:56,124 - IndustrialEvoOptimizer - Best individuals: HallOfFame archive fitness (1): ['<rmse=1.881 node_number=0.300>']
2024-04-03 16:19:56,125 - IndustrialEvoOptimizer - Next population size: 21; max graph depth: 6
2024-04-03 16:20:03,661 - IndustrialDispatcher - Number of used CPU's: 2
2024-04-

Exception ignored in: <function _WeakKeyDictionary.__setitem__.<locals>.on_destroy at 0x000001B17545D1F0>
Traceback (most recent call last):
  File "D:\WORK\Repo\Industiral\IndustrialTS\venv_3.9_new\lib\site-packages\joblib\_dask.py", line 82, in on_destroy
    del self._data[key]
KeyError: 1861470264176
Exception ignored in: <function _WeakKeyDictionary.__setitem__.<locals>.on_destroy at 0x000001B1759295E0>
Traceback (most recent call last):
  File "D:\WORK\Repo\Industiral\IndustrialTS\venv_3.9_new\lib\site-packages\joblib\_dask.py", line 82, in on_destroy
    del self._data[key]
KeyError: 1857737900368
Exception ignored in: <function _WeakKeyDictionary.__setitem__.<locals>.on_destroy at 0x000001B17FB52C10>
Traceback (most recent call last):
  File "D:\WORK\Repo\Industiral\IndustrialTS\venv_3.9_new\lib\site-packages\joblib\_dask.py", line 82, in on_destroy
    del self._data[key]
KeyError: 1861689906832
Exception ignored in: <function _WeakKeyDictionary.__setitem__.<locals>.on_destroy

2024-04-03 16:21:14,862 - full garbage collections took 12% CPU time recently (threshold: 10%)
2024-04-03 16:21:25,667 - full garbage collections took 24% CPU time recently (threshold: 10%)
2024-04-03 16:21:32,243 - full garbage collections took 26% CPU time recently (threshold: 10%)
2024-04-03 16:21:37,849 - full garbage collections took 26% CPU time recently (threshold: 10%)
2024-04-03 16:21:44,251 - full garbage collections took 26% CPU time recently (threshold: 10%)
2024-04-03 16:21:53,145 - full garbage collections took 26% CPU time recently (threshold: 10%)
2024-04-03 16:22:02,245 - full garbage collections took 28% CPU time recently (threshold: 10%)
2024-04-03 16:22:18,763 - full garbage collections took 28% CPU time recently (threshold: 10%)
2024-04-03 16:22:22,572 - full garbage collections took 27% CPU time recently (threshold: 10%)
2024-04-03 16:22:34,246 - full garbage collections took 28% CPU time recently (threshold: 10%)
2024-04-03 16:22:43,814 - full garbage collections

...

Generations:   0%|          | 2/10000 [13:36<1165:20:58, 419.61s/gen]

2024-04-03 16:31:06,742 - IndustrialEvoOptimizer - Generation num: 4 size: 34
2024-04-03 16:31:06,742 - IndustrialEvoOptimizer - Best individuals: HallOfFame archive fitness (1): ['<rmse=1.784 node_number=0.400>']
2024-04-03 16:31:06,743 - GroupedCondition - Optimisation stopped: Time limit is reached


Generations:   0%|          | 2/10000 [13:36<1134:20:06, 408.44s/gen]

2024-04-03 16:31:06,747 - OptimisationTimer - Composition time: 13.615 min
2024-04-03 16:31:06,748 - OptimisationTimer - Algorithm was terminated due to processing time limit
2024-04-03 16:31:06,753 - IndustrialEvoOptimizer - Generation num: 5 size: 1
2024-04-03 16:31:06,755 - IndustrialEvoOptimizer - Best individuals: HallOfFame archive fitness (1): ['<rmse=1.784 node_number=0.400>']
2024-04-03 16:31:06,756 - IndustrialEvoOptimizer - no improvements for 1 iterations
2024-04-03 16:31:06,757 - IndustrialEvoOptimizer - spent time: 13.6 min
2024-04-03 16:31:06,762 - GPComposer - GP composition finished
2024-04-03 16:31:06,765 - DataSourceSplitter - Stratificated splitting of data is disabled.
2024-04-03 16:31:06,766 - DataSourceSplitter - Hold out validation is applied.
2024-04-03 16:31:06,773 - ApiComposer - Hyperparameters tuning started with 1 min. timeout
2024-04-03 16:31:06,780 - SimultaneousTuner - Hyperparameters optimization start: estimation of metric for initial graph


2024-04-03 16:31:13,043 - SimultaneousTuner - Initial graph: {'depth': 4, 'length': 4, 'nodes': [treg, channel_filtration, quantile_extractor, scaling]}
treg - {}
channel_filtration - {'distance': 'euclidean', 'centroid_metric': 'euclidean', 'sample_metric': 'chebyshev', 'selection_strategy': 'sum'}
quantile_extractor - {'stride': 3, 'window_size': 21}
scaling - {} 
Initial metric: [2.906]
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]2024-04-03 16:31:13,099 - build_posterior_wrapper took 0.017987 seconds
2024-04-03 16:31:13,102 - TPE using 0 trials
 10%|█         | 1/10 [00:01<00:10,  1.13s/trial, best loss: 2.421132291609469]2024-04-03 16:31:14,229 - build_posterior_wrapper took 0.017986 seconds
2024-04-03 16:31:14,230 - TPE using 1/1 trials with best loss 2.421132
 20%|██        | 2/10 [00:02<00:09,  1.13s/trial, best loss: 2.421132291609469]2024-04-03 16:31:15,357 - build_posterior_wrapper took 0.016954 seconds
2024-04-03 16:31:15,358 - TPE using 2/2 trials with best loss 2

In [44]:
auto_labels = industrial_auto_model.predict(test_data)
auto_metrics = industrial_auto_model.get_metrics(target=test_data[1],
                                 rounding_order=3,
                                 metric_names=('r2', 'rmse', 'mae'))

In [12]:
industrial_auto_model.solver.current_pipeline.show()
industrial_auto_model.plot_operation_distribution(mode='each')
industrial_auto_model.plot_fitness_by_generation()

2024-04-03 17:59:52,008 - OperationsAnimatedBar - Visualizing optimization history... It may take some time, depending on the history size.
2024-04-03 17:59:52,289 - MovieWriter ffmpeg unavailable; using Pillow instead.
2024-04-03 17:59:52,289 - Animation.save using <class 'matplotlib.animation.PillowWriter'>
2024-04-03 18:00:02,444 - OperationsAnimatedBar - The animation was saved to "D:\WORK\Repo\Industiral\IndustrialTS\results_of_experiments\history_animated_bars.gif".
2024-04-03 18:00:02,446 - FitnessBox - Visualizing optimization history... It may take some time, depending on the history size.
2024-04-03 18:00:02,535 - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2024-04-03 18:00:02,544 - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the a

In [45]:
auto_metrics

,r2,rmse,mae
0,0.612,1.121,0.756


In [46]:
predictions = np.vstack([test_data[1].flatten(),labels.flatten(),auto_labels.flatten()]).T

In [47]:
all_prediction = pd.DataFrame(predictions,columns=['target','baseline','automl'])

In [55]:
all_prediction.plot()
plt.show()

## Compare with State of Art (SOTA) models

In [49]:
df = pd.read_csv(data_path+'/ts_regression_sota_results.csv',sep=';')
df = df[df['ds/type'] == dataset_name].iloc[:,:25]
df.index = df['algorithm']
df = df.drop(['algorithm','ds/type'], axis=1)
df = df.replace(',','.', regex=True).astype(float)

In [51]:
df['Fedot_Industrial_tuned'] = metrics['rmse'][0]
df['Fedot_Industrial_AutoML'] = auto_metrics['rmse'][0]
df = df.T

In [52]:
df.sort_values('min')['min']

FreshPRINCE_RMSE           1.020426
Fedot_Industrial_tuned     1.032000
DrCIF_RMSE                 1.046399
RotF_RMSE                  1.097089
Fedot_Industrial_AutoML    1.121000
TSF_RMSE                   1.172730
RandF_RMSE                 1.182821
XGBoost_RMSE               1.272832
RIST_RMSE                  1.301506
RDST_RMSE                  1.382981
MultiROCKET_RMSE           1.454406
SingleInception_RMSE       1.490343
InceptionT_RMSE            1.542210
5NN-DTW_RMSE               1.593567
CNN_RMSE                   1.602674
FPCR-Bs_RMSE               1.607024
ResNet_RMSE                1.610736
FCN_RMSE                   1.636335
FPCR_RMSE                  1.637966
Grid-SVR_RMSE              1.699988
Ridge_RMSE                 1.713667
ROCKET_RMSE                1.856612
5NN-ED_RMSE                1.905470
1NN-ED_RMSE                2.129638
1NN-DTW_RMSE               2.211915
Name: min, dtype: float64

In [53]:
df.sort_values('max')['max']

Fedot_Industrial_tuned     1.032000
Fedot_Industrial_AutoML    1.121000
RIST_RMSE                  1.887153
RotF_RMSE                  2.088333
FreshPRINCE_RMSE           2.101253
MultiROCKET_RMSE           2.137563
TSF_RMSE                   2.181236
RDST_RMSE                  2.200364
DrCIF_RMSE                 2.201153
ResNet_RMSE                2.404590
RandF_RMSE                 2.446314
5NN-DTW_RMSE               2.502464
XGBoost_RMSE               2.511297
FPCR-Bs_RMSE               2.512605
FPCR_RMSE                  2.523426
5NN-ED_RMSE                2.554152
FCN_RMSE                   2.559661
InceptionT_RMSE            2.574168
SingleInception_RMSE       2.633115
CNN_RMSE                   2.674200
Grid-SVR_RMSE              2.800419
ROCKET_RMSE                2.870444
Ridge_RMSE                 2.904762
1NN-ED_RMSE                3.281798
1NN-DTW_RMSE               3.408802
Name: max, dtype: float64

In [54]:
df.sort_values('average')['average']

Fedot_Industrial_tuned     1.032000
Fedot_Industrial_AutoML    1.121000
FreshPRINCE_RMSE           1.490442
RIST_RMSE                  1.501047
RotF_RMSE                  1.559385
DrCIF_RMSE                 1.594442
TSF_RMSE                   1.684828
RandF_RMSE                 1.708196
XGBoost_RMSE               1.716903
RDST_RMSE                  1.772813
MultiROCKET_RMSE           1.773578
ResNet_RMSE                1.938074
InceptionT_RMSE            2.030315
FPCR_RMSE                  2.052389
5NN-DTW_RMSE               2.055256
FCN_RMSE                   2.069046
FPCR-Bs_RMSE               2.097964
SingleInception_RMSE       2.149368
CNN_RMSE                   2.150854
Grid-SVR_RMSE              2.203537
5NN-ED_RMSE                2.251424
ROCKET_RMSE                2.275254
Ridge_RMSE                 2.363609
1NN-DTW_RMSE               2.742105
1NN-ED_RMSE                2.822595
Name: average, dtype: float64